In [2]:
import pandas as pd
import numpy as np
import os
from ete3 import NCBITaxa, PhyloTree
ncbi = NCBITaxa()
from collections import Counter
import gseapy as gp
from gseapy.plot import barplot, dotplot

In [4]:
table_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Tables\\'
fig_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Figures\\'
samples=pd.read_excel(table_path+'Table S1\\Table S1.TE_Evo_352_Species.xlsx')
samples.shape

(352, 7)

### GO Enrichment of TE-associated Genes

In [5]:
# prepare GO information
goOnto=pd.read_excel('D:\\18.TE_Evolution\\03.Protein_Annotation\\GO_Titles.xlsx').fillna("")
goOnto.columns=['GO_Term','GO_Type',"Title","Level"]
GO_Term_dic={}
for i in range(goOnto.shape[0]):
    go_term=goOnto.loc[i,'GO_Term'].replace(".",":")
    GO_Term_dic[go_term]={}
    for col in ['GO_Type',"Title","Level"]:
        GO_Term_dic[go_term][col]=goOnto.loc[i,col]

In [9]:
go_path='D:\\18.TE_Evolution\\03.Protein_Annotation\\GO\\'
gene_sets={}
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+".wego.csv" in os.listdir(go_path):
        df=pd.read_csv(go_path+specie_id+".wego.csv")
        df=df.dropna().reset_index(drop=True)
        for i in range(df.shape[0]):
            gene=df.loc[i,'Gene']
            gos=df.loc[i,'Go_Terms']
            if "\t" in gos:
                gos=gos.split("\t")
            elif " " in gos:
                gos=gos.split(" ")
            else:
                gos=[gos]
            for go in gos:
                if go in gene_sets:
                    gene_sets[go].append(gene)
                else:
                    gene_sets[go]=[gene]

0
100
200
300


In [10]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part2.TE-associated_Genetic_Functions\\'
TEpep_genes_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
TE_dupgene_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
gene_list=[]
count=0
for  i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+".TEpep_to_Gene.txt" in os.listdir(TEpep_genes_path):
        gene_list+= [gene.strip() for gene in open(TEpep_genes_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]
    if specie_id+".TE_DupGenes.txt" in os.listdir(TE_dupgene_path):
        gene_list+=[gene.strip() for gene in open(TE_dupgene_path+specie_id+".TE_DupGenes.txt",'r').readlines()]
gene_list=list(set(gene_list))
count=len(gene_list)
print('TE-associated Genes',count)
if len(gene_list)>10:
    try:
        enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
            outdir=save_path,#输出目录
            top_term=20,
            cutoff=0.01#pvalue阈值
                                             )
        result=enr.results
        result=result.sort_values("Adjusted P-value")
        for col in ['GO_Type',"Title","Level"]:
            result[col]=result['Term'].apply(lambda x:GO_Term_dic[x][col] if x in GO_Term_dic else '')
        print(result.shape)
        result.to_excel(save_path+"TE-associated_Genes_GO_Enrichment.xlsx",index=False)
    except:
        pass
else:
    print('Error: No TE-associated genes!')

TE-associated Genes 5871053


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


(14507, 11)


In [11]:
result[result["Adjusted P-value"]<0.01].shape

(591, 11)

### GO Enrichment of Transpoase Domesticated Genes

In [12]:
go_path='D:\\18.TE_Evolution\\03.Protein_Annotation\\GO\\'
TEpep_genes_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
gene_sets={}
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+".TEpep_to_Gene.txt"
    bg_genes={}
    if bg_file in os.listdir(TEpep_genes_path):
        for gene in [gene.strip() for gene in open(TEpep_genes_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]:
            bg_genes[gene]=1
    if specie_id+".wego.csv" in os.listdir(go_path):
        df=pd.read_csv(go_path+specie_id+".wego.csv")
        df=df.dropna().reset_index(drop=True)
        for i in range(df.shape[0]):
            gene=df.loc[i,'Gene']
            if gene in bg_genes:
                gos=df.loc[i,'Go_Terms']
                if "\t" in gos:
                    gos=gos.split("\t")
                elif " " in gos:
                    gos=gos.split(" ")
                else:
                    gos=[gos]
                for go in gos:
                    if go in gene_sets:
                        gene_sets[go].append(gene)
                    else:
                        gene_sets[go]=[gene]

0
100
200
300


In [13]:
gene_list_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Super_Family\\'
families=list(set([f.split(".TEpep")[0][:-5] for f in os.listdir(gene_list_path)]))
families

['CACTA',
 'Helitron',
 'LINE',
 'Gypsy',
 'piggybac',
 'Crypton',
 'hAT',
 'ISC1316',
 'Copia',
 'mariner_ant1',
 'DDE_1',
 'P_element',
 'ISb',
 'Mariner',
 'Mutator',
 'LTR_Roo']

In [14]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
for family in families:
    gene_list=[]
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        g_file=family+"_"+specie_id+'.TEpep_to_Gene.txt'#".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            gene_list+=genes
    print(family,len(gene_list))
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            for col in ['GO_Type',"Title","Level"]:
                result[col]=result['Term'].apply(lambda x:GO_Term_dic[x][col] if x in GO_Term_dic else '')
            print(family,result.shape)
            result.to_excel(save_path+family+"_GO_TEpep_Dom_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)

CACTA 54024


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


CACTA (1927, 11)
Helitron 16128


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Helitron (900, 11)
LINE 168598


...

Copia (5278, 11)
mariner_ant1 45
mariner_ant1 (9, 11)
DDE_1 1239
DDE_1 (219, 11)
P_element 2
P_element
ISb 2
ISb
Mariner 744
Mariner (100, 11)
Mutator 75681


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Mutator (1782, 11)
LTR_Roo 4001


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


LTR_Roo (618, 11)


### Enrichment of TE induced Dup Genes

In [15]:
bg_genes={}
gene_sets={}
background_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+'.TE_DupGenes.txt'#".TE_DupGenes.txt"
    #rm_file=specie_id+".TEpep_to_Gene.txt"
    if bg_file in os.listdir(background_path):
        genes=[gene.strip() for gene in open(background_path+bg_file,'r').readlines()]
        for gene in genes:
            bg_genes[gene]=1
        if specie_id+".wego.csv" in os.listdir(go_path):
            df=pd.read_csv(go_path+specie_id+".wego.csv")
            df=df.dropna().reset_index(drop=True)
            for i in range(df.shape[0]):
                gene=df.loc[i,'Gene']
                if gene in bg_genes:
                    gos=df.loc[i,'Go_Terms']
                    if "\t" in gos:
                        gos=gos.split("\t")
                    elif " " in gos:
                        gos=gos.split(" ")
                    else:
                        gos=[gos]
                    for go in gos:
                        if go in gene_sets:
                            gene_sets[go].append(gene)
                        else:
                            gene_sets[go]=[gene]

0
100
200
300


In [16]:
gene_list_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Super_Family\\'
families=list(set([f.split(".TE")[0][:-5] for f in os.listdir(gene_list_path)]))
families

['CACTA',
 'Helitron',
 'Gypsy',
 'PIF-Harbinger',
 'hAT',
 'Copia',
 'Mutator',
 'Tc1_Mariner']

In [17]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
tepep_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
for family in families:
    gene_list=[]
    count=0
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        tepep_genes={}
        if specie_id+".TEpep_to_Gene.txt" in os.listdir(tepep_path):
            for gene in [gene.strip() for gene in open(tepep_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]:
                tepep_genes[gene]=1
        g_file=family+"_"+specie_id+".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            for gene in genes:
                if gene not in tepep_genes:
                    gene_list.append(gene)
                else:
                    count+=1
    print(family,len(gene_list),'TEpep Genes',count)
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            for col in ['GO_Type',"Title","Level"]:
                result[col]=result['Term'].apply(lambda x:GO_Term_dic[x][col] if x in GO_Term_dic else '')
            print(family,result.shape)
            result.to_excel(save_path+family+"_GO_TE_DupGenes_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)

CACTA 2034450 TEpep Genes 151124


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


CACTA (12525, 11)
Helitron 3233061 TEpep Genes 153108


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Helitron (13798, 11)
Gypsy 1478296 TEpep Genes 266960


...

Copia (10114, 11)
Mutator 2874488 TEpep Genes 194916


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Mutator (13277, 11)
Tc1_Mariner 1514834 TEpep Genes 63659


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Tc1_Mariner (11919, 11)
